# **Multi-Wavelength Algorithm Tavora et al. (2020)**

This notebook provides a Python implementation of the multi-wavelength algorithm for Suspended Particulate Matter detection, as proposed by Tavora et al. (2020). The original paper can be found at: https://www.mdpi.com/2072-4292/12/13/2172.

This adaptation uses numpy and rasterio to process Sentinel-3 OLCI remote sensing reflectance products derived from ACOLITE. Nevertheless, the variables and thresholds can be adjusted for other sensors.

## Import and/or install all necessary libraries

### Install libraries if using Google Colab

In [1]:
!pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.7 MB/s eta 0:00:00


In [2]:
!pip install openeo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.9 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.7.1
    Uninstalling xarray-2025.7.1:
      Successfully uninstalled xarray-2025.7.1


In [3]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 51.6 MB/s eta 0:00:00


### Import libraries

In [4]:
from netCDF4 import Dataset
from scipy import ndimage
import numpy as np
import os
import xarray as xr
import pandas as pd
import geopandas as gpd
from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import interp1d
from scipy.ndimage import generic_filter
from scipy.ndimage import zoom
from sklearn.decomposition import PCA
import ee
import requests
from datetime import datetime, timedelta
import time
import openeo
import rasterio
import io

## Create connection to Earth Observation repositories

You must hold an OpenEO/Copernicus DataSpace Ecosystem or Google Earth Engine account depending on the temperature product you wish to use.

In [5]:
# --- CONFIGURATION ---
# Set your desired data source: 'GEE' or 'OPENEO'
DATA_SOURCE = 'OPENEO'

# If using GEE, specify your project name here
GEE_PROJECT_NAME = 'your_project_name'
# ---------------------

In [6]:
# --- CONNECTION LOGIC ---
print(f"Attempting to connect to {DATA_SOURCE}...")

if DATA_SOURCE == 'GEE':
    try:
        import ee
        ee.Authenticate()
        ee.Initialize(project=GEE_PROJECT_NAME)
        print("✅ Successfully connected to Google Earth Engine.")
    except Exception as e:
        print(f"❌ Failed to connect to GEE. Error: {e}")

elif DATA_SOURCE == 'OPENEO':
    try:
        import openeo
        # Connect to the openEO backend
        connection = openeo.connect("openeofed.dataspace.copernicus.eu")
        # Authenticate using OIDC device code flow
        connection.authenticate_oidc()
        print("✅ Successfully connected to openEO.")
    except Exception as e:
        print(f"❌ Failed to connect to openEO. Error: {e}")

else:
    print("❌ Invalid DATA_SOURCE. Please choose 'GEE' or 'OPENEO'.")

Attempting to connect to OPENEO...


Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=LZJK-QCRT 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.
✅ Successfully connected to openEO.


## Define functions

In [7]:
#water-leaving reflectance to ato below water remote sensing reflectance
# Lee et al., 2002 approach to transfer above water remote sensing
def RRS2rrs(RWRS):
    rrs = RWRS /  (0.52 + (1.70 * RWRS))
    return rrs

In [8]:
def aswcorrection(temp, nm_touse):
#-------------------------------------------------------------------------
#   Corrects water absorption to the temperature Rrs(lambda) was measured
#
# INPUTS:
#
#   temp      - temperature at which rrs was measured in Celsius
#   nm_touse  - wavelengths (or bands)
#
# OUTPUTS:
#
#  asw_Tcorr - water absorption corrected to temperature rrs was measured
#
# This function used water absorption of pure waters (asw) measured by
# from Pope and Fry (1997) and Kou et al (1993).
# We account for the temperature dependence using the Taylor series of
# Sulivan et al (2006), applying the temperature coefficients by Rottgers
# et al (2014)
#
#--------------------------------------------------------------------------
    absorp = {}
    nm_sample = [num for num in range(600, 2501)]
    # absorption values every 1nm (wavelength)
    asw  = [0.22240,0.23224,0.24208,0.24914,0.25342,0.25770,0.25978,0.26186,0.26320,0.26380,0.26440,0.26524,0.26608,0.26676,0.26728,0.26780,0.26896,0.27012,0.27166,0.27358,0.27550,0.27770,0.27990,0.28148,0.28244,0.28340,0.28620,0.289,0.29064,0.29112,0.29160,0.29476,0.29792,0.29984,0.30052,0.30120,0.30380,0.30640,0.30832,0.30956,0.31080,0.31528,0.31976,0.32260,0.32380,0.325,0.329,0.333,0.336,0.338,0.34,0.34720,0.35440,0.36060,0.36580,0.371,0.37980,0.38860,0.39640,0.40320,0.41,0.41560,0.42120,0.425,0.427,0.429,0.43180,0.45,0.45155,0.45560,0.45675,0.45915,0.46150,0.463,0.46450,0.46695,0.472,0.47420,0.47740,0.48060,0.48290,0.487,0.48920,0.49370,0.49770,0.50165,0.50630,0.51190,0.51815,0.52330,0.53480,0.54430,0.55280,0.56125,0.57135,0.58055,0.59305,0.60550,0.61875,0.63190,0.64590,0.683,0.69810,0.715,0.73540,0.75220,0.77250,0.79270,0.81470,0.84010,0.86550,0.89790,0.93010,0.96940,1.0067,1.0492,1.0952,1.1410,1.1901,1.2427,1.268850000,1.2950,1.3489,1.4061,1.4667,1.5253,1.5924,1.6646,1.7607,1.8444,1.9624,2.0801,2.1974,2.3144,2.4482,2.5304,2.6123,2.6428,2.6733,2.7377,2.7680,2.7982,2.8113,2.8245,2.8376,2.8338,2.8468,2.8430,2.8560,2.8522,2.8484,2.8613,2.8575,2.8704,2.8666,2.8794,2.8756,2.871850000,2.8681,2.8643,2.8605,2.8733,2.8695,2.8657,2.861950000,2.8582,2.8545,2.8344,2.8307,2.8270,2.8234,2.8116,2.7998,2.7799,2.7763,2.7565,2.7529,2.7413,2.7297,2.7101,2.6905,2.6549,2.643450000,2.6320,2.6127,2.5933,2.5580,2.5229,2.5117,2.5005,2.4656,2.4465,2.4117,2.392850000,2.3740,2.3552,2.3365,2.3178,2.299150000,2.2805,2.2462,2.2434,2.2328,2.2222,2.2038,2.2011,2.198350000,2.1956,2.1773,2.1902,2.1875,2.192550000,2.1976,2.2103,2.2230,2.2357,2.2484,2.2764,2.3043,2.316850000,2.3294,2.3878,2.4460,2.481150000,2.5163,2.6199,2.7689,2.8414,2.9139,3.1075,3.202050000,3.2966,3.4588,3.6055,3.653850000,3.7022,3.7879,3.8209,3.8539,3.9092,3.9494,3.974550000,3.9997,4.0397,4.057250000,4.0748,4.1145,4.1319,4.1493,4.2036,4.1986,4.2232,4.247750000,4.2723,4.289350000,4.3064,4.3601,4.369650000,4.3792,4.4326,4.449350000,4.4661,4.5047,4.528550000,4.5524,4.6052,4.6580,4.6816,4.7052,4.728650000,4.7521,4.8332,4.863650000,4.8941,4.9604,5.004950000,5.0495,5.1154,5.1596,5.2038,5.2979,5.341750000,5.3856,5.4649,5.5155,5.5661,5.6094,5.6527,5.7454,5.7884,5.8314,5.9235,5.966150000,6.0088,6.0512,6.0936,6.1850,6.2271,6.2692,6.3601,6.4088,6.4575,6.5618,6.6171,6.6724,6.7205,6.7686,6.8997,6.968150000,7.0366,7.111650000,7.1867,7.3579,7.459850000,7.5618,7.6770,7.7922,8.0852,8.247350000,8.4095,8.605050000,8.8006,9.2680,9.5436,9.8192,10.12070000,10.42220000,10.77660000,11.131,11.94410000,12.37670000,12.80930000,13.27375000,13.73820000,14.19390000,14.64960000,15.57370000,16.22695000,16.88020000,17.53065000,18.18110000,19.63080000,20.34290000,21.055,21.96355000,22.87210000,23.77685000,24.68160000,25.84735000,27.01310000,29.76250000,31.24920000,32.73590000,34.47985000,36.22380000,37.76330000,39.30280000,40.57350000,41.84420000,42.84780000,43.85140000,45.37490000,45.84975000,46.32460000,46.66725000,47.00990000,47.28620000,47.56250000,47.70795000,47.85340000,47.99820000,48.143,48.28725000,48.43150000,48.63990000,48.65440000,48.66890000,48.61915000,48.56940000,48.39160000,48.21380000,48.03675000,47.85970000,47.61950000,47.37930000,47.07635000,46.77340000,46.40810000,46.04280000,45.67895000,45.31510000,44.82605000,44.337,43.97635000,43.61570000,43.13030000,42.64490000,42.16145000,41.678,41.19650000,40.715,40.17275000,39.63050000,39.15295000,38.67540000,38.13730000,37.59920000,37.00090000,36.40260000,35.80665000,35.21070000,34.61715000,34.02360000,33.49430000,32.965,32.43785000,31.91070000,31.38555000,30.86040000,30.21415000,29.56790000,28.98565000,28.40340000,27.82345000,27.24350000,26.72705000,26.21060000,25.635,25.05940000,24.48605000,23.91270000,23.60570000,23.29870000,22.99170000,22.48385000,21.976,21.53065000,21.08530000,20.64170000,20.19810000,19.81655000,19.435,19.05495000,18.67490000,18.4167666666667,18.1586333333333,17.90050000,17.52370000,17.14690000,16.89120000,16.63550000,16.38075000,16.126,15.87225000,15.61850000,15.4848666666667,15.3512333333333,15.21760000,15.08455000,14.95150000,14.75975000,14.568,14.49520000,14.42240000,14.3695666666667,14.3167333333333,14.26390000,14.19170000,14.11950000,14.10630000,14.09310000,14.0799666666667,14.0668333333333,14.05370000,14.09910000,14.14450000,14.24805000,14.35160000,14.37710000,14.40260000,14.42810000,14.58895000,14.74980000,14.96805000,15.18630000,15.3264666666667,15.4666333333333,15.60680000,15.82325000,16.03970000,16.31295000,16.58620000,16.76250000,16.93880000,17.11510000,17.44345000,17.77180000,18.21340000,18.655,18.8663333333333,19.0776666666667,19.289,19.67025000,20.05150000,20.29850000,20.54550000,20.79250000,21.22735000,21.66220000,21.9443333333333,22.2264666666667,22.50860000,22.940,23.37140000,23.4255333333333,23.4796666666667,23.53380000,24.01855000,24.50330000,24.9298333333333,25.3563666666667,25.78290000,26.48730000,27.19170000,27.7627666666667,28.3338333333333,28.90490000,30.27180000,31.63870000,32.8699666666667,34.1012333333333,35.33250000,37.85230000,40.37210000,42.62270000,44.87330000,47.12390000,49.9878333333333,52.8517666666667,55.71570000,60.45770000,65.19970000,68.5845666666667,71.9694333333333,75.35430000,80.66160000,85.96890000,88.9563666666667,91.9438333333333,94.93130000,97.9759666666667,101.020633333333,104.0653000,107.0241500,109.9830000,111.3361000,112.6892000,114.0423000,115.027266666667,116.012233333333,116.9972000,117.9780000,118.9588000,119.216233333333,119.473666666667,119.7311000,119.9872000,120.2433000,120.4994000,120.754233333333,121.009066666667,121.2639000,122.2318000,123.1997000,123.094933333333,122.990166666667,122.8854000,123.136466666667,123.387533333333,123.6386000,123.888366666667,124.138133333333,124.3879000,124.8135500,125.2392000,125.486566666667,125.733933333333,125.9813000,126.227433333333,126.473566666667,126.7197000,126.9646000,127.2095000,127.4544000,127.698066666667,127.941733333333,128.1854000,128.078466666667,127.971533333333,127.8646000,127.2350000,126.6054000,126.848133333333,127.090866666667,127.3336000,126.881066666667,126.428533333333,125.9760000,125.8719000,125.7678000,125.6637000,125.214766666667,124.765833333333,124.3169000,124.214666666667,124.112433333333,124.0102000,123.221233333333,122.432266666667,121.6433000,121.543766666667,121.444233333333,121.3447000,120.9037000,120.4627000,120.0217000,119.923933333333,119.826166666667,119.7284000,119.290866666667,118.853333333333,118.4158000,117.9804000,117.5450000,117.1096000,116.6763000,116.2430000,115.8097000,115.7163000,115.6229000,115.5295000,115.099566666667,114.669633333333,114.2397000,113.811833333333,113.383966666667,112.9561000,112.6144250,112.2727500,111.9310750,111.5894000,111.166033333333,110.742666666667,110.3193000,110.231466666667,110.143633333333,110.0558000,109.968366666667,109.880933333333,109.7935000,109.3746000,108.9557000,108.5368000,108.4510000,108.3652000,108.2794000,108.1940750,108.1087500,108.0234250,107.9381000,108.182533333333,108.426966666667,108.6714000,109.243233333333,109.815066666667,110.3869000,110.6283000,110.8697000,111.1111000,111.7596000,112.4081000,113.0566000,113.7051000,114.594533333333,115.483966666667,116.3734000,116.933466666667,117.493533333333,118.0536000,118.9348750,119.8161500,120.6974250,121.5787000,122.778633333333,123.978566666667,125.1785000,126.695633333333,128.212766666667,129.7299000,131.0780000,132.4261000,133.7742000,135.1223000,137.2656000,139.4089000,141.5522000,144.005933333333,146.459666666667,148.9134000,151.1944000,153.4754000,155.7564000,158.0374000,161.102633333333,164.167866666667,167.2331000,170.2026500,173.1722000,176.1417500,179.1113000,183.0949000,187.0785000,191.0621000,194.4718250,197.8815500,201.2912750,204.7010000,209.9051000,215.1092000,220.3133000,224.1545750,227.9958500,231.8371250,235.6784000,241.772633333333,247.866866666667,253.9611000,257.9916250,262.0221500,266.0526750,270.0832000,274.7909750,279.4987500,284.2065250,288.9143000,295.852166666667,302.790033333333,309.7279000,313.2192500,316.7106000,320.2019500,323.6933000,327.6277500,331.5622000,335.4966500,339.4311000,344.7333000,350.0355000,355.3377000,359.6900250,364.0423500,368.3946750,372.7470000,378.4537250,384.1604500,389.8671750,395.5739000,404.464166666667,413.354433333333,422.2447000,432.6913500,443.1380000,453.5846500,464.0313000,477.6113250,491.1913500,504.7713750,518.3514000,538.4498500,558.5483000,578.6467500,598.7452000,629.1617500,659.5783000,689.9948500,720.4114000,773.0067000,825.6020000,878.1973000,933.9887000,989.7801000,1045.571500,1101.362900,1174.837950,1248.313000,1321.788050,1395.263100,1470.562300,1545.861500,1621.160700,1696.459900,1764.617250,1832.774600,1900.931950,1969.089300,2034.628150,2100.167000,2165.705850,2231.244700,2289.737425,2348.230150,2406.722875,2465.215600,2507.847425,2550.479250,2593.111075,2635.742900,2669.284950,2702.827000,2736.369050,2769.911100,2805.470925,2841.030750,2876.590575,2912.150400,2940.911000,2969.671600,2998.432200,3027.192800,3047.017325,3066.841850,3086.666375,3106.490900,3125.316780,3144.142660,3162.968540,3181.794420,3200.620300,3207.124300,3213.628300,3220.132300,3226.600280,3233.068260,3239.536240,3246.004220,3252.472200,3256.731900,3260.991600,3265.251300,3269.511000,3271.585125,3273.659250,3275.733375,3277.807500,3275.557800,3273.308100,3271.058400,3268.808700,3263.134720,3257.460740,3251.786760,3246.112780,3240.438800,3218.941675,3197.444550,3175.947425,3154.450300,3130.933050,3107.415800,3083.898550,3060.381300,3036.991675,3013.602050,2990.212425,2966.822800,2937.627920,2908.433040,2879.238160,2850.043280,2820.848400,2795.237440,2769.626480,2744.015520,2718.404560,2692.793600,2663.537200,2634.280800,2605.024400,2575.768000,2534.034750,2492.301500,2450.568250,2408.835000,2385.400540,2361.966080,2338.531620,2315.097160,2291.662700,2258.740950,2225.819200,2192.897450,2159.975700,2136.846520,2113.717340,2090.588160,2067.458980,2044.329800,2013.848575,1983.367350,1952.886125,1922.404900,1899.569940,1876.734980,1853.900020,1831.065060,1808.230100,1782.221325,1756.212550,1730.203775,1704.195000,1684.936340,1665.677680,1646.419020,1627.160360,1607.901700,1593.700200,1579.498700,1565.297200,1551.095700,1536.894200,1517.447850,1498.001500,1478.555150,1459.108800,1443.453040,1427.797280,1412.141520,1396.485760,1380.830000,1360.389220,1339.948440,1319.507660,1299.066880,1278.626100,1266.435180,1254.244260,1242.053340,1229.862420,1217.671500,1204.741600,1191.811700,1178.881800,1165.951900,1152.289620,1138.627340,1124.965060,1111.302780,1097.640500,1087.283740,1076.926980,1066.570220,1056.213460,1045.856700,1035.566000,1025.275300,1014.984600,1004.693900,994.4032000,985.7767600,977.1503200,968.5238800,959.8974400,951.2710000,944.2929600,937.3149200,930.3368800,923.3588400,916.3808000,909.4468800,902.5129600,895.5790400,888.6451200,881.7112000,874.8209600,867.9307200,861.0404800,854.1502400,847.26,840.4130400,833.5660800,826.7191200,819.8721600,813.0252000,807.7948600,802.5645200,797.3341800,792.1038400,786.8735000,781.6758200,776.4781400,771.2804600,766.0827800,760.8851000,756.3453400,751.8055800,747.2658200,742.7260600,738.1863000,734.4542400,730.7221800,726.9901200,723.2580600,719.5260000,716.1041000,712.6822000,709.2603000,705.8384000,702.4165000,698.9946000,695.3120000,691.6294000,687.9468000,684.2642000,680.5816000,677.6935000,674.8054000,671.9173000,669.0292000,666.1411000,663.298083333333,660.455066666667,657.6120500,654.769033333333,651.926016666667,649.0830000,646.8362000,644.5894000,642.3426000,640.0958000,637.8490000,635.6022000,633.3990750,631.1959500,628.9928250,626.7897000,625.013566666667,623.237433333333,621.4613000,619.685166666667,617.909033333333,616.1329000,613.9383000,611.7437000,609.5491000,607.3545000,605.1599000,604.039016666667,602.918133333333,601.7972500,600.676366666667,599.555483333333,598.4346000,597.1702600,595.9059200,594.6415800,593.3772400,592.1129000,591.382966666667,590.653033333333,589.9231000,589.193166666667,588.463233333333,587.7333000,586.9332400,586.1331800,585.3331200,584.5330600,583.7330000,583.759483333333,583.785966666667,583.8124500,583.838933333333,583.865416666667,583.8919000,583.7942000,583.6965000,583.5988000,583.5011000,583.4034000,583.3057000,585.2931000,585.6778900,586.0626800,586.4474700,586.8322600,587.2170500,587.6018400,587.9866300,588.3714200,588.7562100,589.1410000,589.655033333333,590.169066666667,590.6831000,591.197133333333,591.711166666667,592.2252000,594.4588750,596.6925500,598.9262250,601.1599000,602.7260875,604.2922750,605.8584625,607.4246500,608.9908375,610.5570250,612.1232125,613.6894000,616.0061000,618.3228000,620.6395000,622.9562000,625.2729000,627.5896000,630.374483333333,633.159366666667,635.9442500,638.729133333333,641.514016666667,644.2989000,646.3925900,648.4862800,650.5799700,652.6736600,654.7673500,656.8610400,658.9547300,661.0484200,663.1421100,665.2358000,678.5177000,691.7996000,697.514483333333,703.229366666667,708.9442500,714.659133333333,720.374016666667,726.0889000,731.645266666667,737.201633333333,742.7580000,748.314366666667,753.870733333333,759.4271000,764.9456000,770.4641000,775.9826000,781.5011000,787.0196000,792.5381000,797.2498400,801.9615800,806.6733200,811.3850600,816.0968000,820.8085400,825.5202800,830.2320200,834.9437600,839.6555000,850.3632000,861.0709000,871.7786000,875.997716666667,880.216833333333,884.4359500,888.655066666667,892.874183333333,897.0933000,900.110716666667,903.128133333333,906.1455500,909.162966666667,912.180383333333,915.1978000,915.688871428572,916.179942857143,916.671014285714,917.162085714286,917.653157142857,918.144228571429,918.6353000,918.124383333333,917.613466666667,917.1025500,916.591633333333,916.080716666667,915.5698000,917.0517000,918.5336000,920.0155000,921.4974000,922.9793000,924.4612000,925.9431000,923.9417600,921.9404200,919.9390800,917.9377400,915.9364000,916.3595875,916.7827750,917.2059625,917.6291500,918.0523375,918.4755250,918.8987125,919.3219000,919.966133333333,920.610366666667,921.2546000,921.898833333333,922.543066666667,923.1873000,925.624428571429,928.061557142857,930.498685714286,932.935814285714,935.372942857143,937.810071428571,940.2472000,944.6192000,948.9912000,953.3632000,957.7352000,962.1072000,966.4792000,970.8512000,976.163028571429,981.474857142857,986.786685714286,992.098514285714,997.410342857143,1002.72217142857,1008.034000,1025.59331666667,1043.15263333333,1060.711950,1078.27126666667,1095.83058333333,1113.389900,1131.711000,1150.032100,1168.353200,1186.674300,1204.995400,1223.316500,1241.637600,1259.958700,1278.279800,1337.962240,1397.644680,1457.327120,1517.009560,1576.692000,1659.28031428571,1741.86862857143,1824.45694285714,1907.04525714286,1989.63357142857,2072.22188571429,2154.810200,2300.79494285714,2446.77968571429,2592.76442857143,2738.74917142857,2884.73391428571,3030.71865714286,3176.703400,3330.46247272727,3484.22154545455,3637.98061818182,3791.73969090909,3945.49876363636,4099.25783636364,4253.01690909091,4406.77598181818,4560.53505454545,4714.29412727273,4868.053200,5475.618450,6083.183700,6690.748950,7298.314200,7662.768700,8027.223200,8391.677700,8756.132200,9120.586700,9485.041200,9849.495700,10117.1881857143,10384.8806714286,10652.5731571429,10920.2656428571,11187.9581285714,11455.6506142857,11723.34310,11818.2031166667,11913.0631333333,12007.92315,12102.7831666667,12197.6431833333,12292.50320,12387.3632166667,12482.2232333333,12577.08325,12671.9432666667,12766.8032833333,12861.66330,13061.6535333333,13261.6437666667,13461.63400,13491.8651857143,13522.0963714286,13552.3275571429,13582.5587428571,13612.7899285714,13643.0211142857,13673.25230,13656.03294,13638.81358,13621.59422,13604.37486,13587.15550,13569.93614,13552.71678,13535.49742,13518.27806,13501.05870,13411.03384,13321.00898,13230.98412,13140.95926,13050.93440,12963.8219750,12876.70955,12789.5971250,12702.48470,12615.3722750,12528.25985,12441.1474250,12354.03500,12237.8219857143,12121.6089714286,12005.3959571429,11889.1829428571,11772.9699285714,11656.7569142857,11540.54390,11438.8995375,11337.2551750,11235.6108125,11133.96645,11032.3220875,10930.6777250,10829.0333625,10727.38900,10626.5676750,10525.74635,10424.9250250,10324.10370,10223.2823750,10122.46105,10021.6397250,9920.818400,9820.810150,9720.801900,9620.793650,9520.785400,9420.777150,9320.768900,9220.760650,9120.752400,9029.432950,8938.113500,8846.794050,8755.474600,8664.155150,8572.835700,8481.516250,8390.196800,8307.46188750,8224.726975,8141.99206250,8059.257150,7976.52223750,7893.787325,7811.05241250,7728.317500,7646.24183750,7564.166175,7482.09051250,7400.014850,7317.93918750,7235.863525,7153.78786250,7071.712200,7005.871225,6940.030250,6874.189275,6808.348300,6742.507325,6676.666350,6610.825375,6544.984400,6478.88781250,6412.791225,6346.69463750,6280.598050,6214.50146250,6148.404875,6082.30828750,6016.211700,5956.819800,5897.427900,5838.036000,5778.644100,5719.252200,5659.860300,5600.468400,5541.076500,5488.31041250,5435.544325,5382.77823750,5330.012150,5277.24606250,5224.479975,5171.71388750,5118.947800,5083.581550,5048.215300,5012.849050,4977.482800,4942.116550,4906.750300,4871.384050,4836.017800,4800.651550,4765.285300,4712.892400,4660.499500,4608.106600,4555.713700,4503.320800,4450.927900,4398.535000,4359.23537777778,4319.93575555556,4280.63613333333,4241.33651111111,4202.03688888889,4162.73726666667,4123.43764444444,4084.13802222222,4044.838400,4008.806250,3972.774100,3936.741950,3900.709800,3864.677650,3828.645500,3792.613350,3756.581200,3727.96521111111,3699.34922222222,3670.73323333333,3642.11724444444,3613.50125555556,3584.88526666667,3556.26927777778,3527.65328888889,3499.037300,3472.66981111111,3446.30232222222,3419.93483333333,3393.56734444445,3367.19985555556,3340.83236666667,3314.46487777778,3288.09738888889,3261.729900,3234.744750,3207.759600,3180.774450,3153.789300,3126.804150,3099.819000,3072.833850,3045.848700,3026.28161666667,3006.71453333333,2987.147450,2967.58036666667,2948.01328333333,2928.446200,2908.87911666667,2889.31203333333,2869.744950,2850.17786666667,2830.61078333333,2811.043700,2791.47661666667,2771.90953333333,2752.342450,2732.77536666667,2713.20828333333,2693.641200,2677.93295555556,2662.22471111111,2646.51646666667,2630.80822222222,2615.09997777778,2599.39173333333,2583.68348888889,2567.97524444444,2552.267000,2538.65216666667,2525.03733333333,2511.422500,2497.80766666667,2484.19283333333,2470.578000,2456.96316666667,2443.34833333333,2429.733500,2417.534800,2405.336100,2393.137400,2380.938700,2368.740000,2356.541300,2344.342600,2332.143900,2319.945200,2311.288020,2302.630840,2293.973660,2285.316480,2276.659300,2268.002120,2259.344940,2250.687760,2242.030580,2233.373400,2225.88911111111,2218.40482222222,2210.92053333333,2203.43624444444,2195.95195555556,2188.46766666667,2180.98337777778,2173.49908888889,2166.014800,2159.23505555556,2152.45531111111,2145.67556666667,2138.89582222222,2132.11607777778,2125.33633333333,2118.55658888889,2111.77684444444,2104.997100,2101.154600,2097.312100,2093.469600,2089.627100,2085.784600,2081.942100,2078.099600,2074.257100,2070.414600,2066.572100,2062.764640,2058.957180,2055.149720,2051.342260,2047.534800,2043.727340,2039.919880,2036.112420,2032.304960,2028.497500,2026.942200,2025.386900,2023.831600,2022.276300,2020.721000,2019.165700,2017.610400,2016.055100,2014.499800,2013.589090,2012.678380,2011.767670,2010.856960,2009.946250,2009.035540,2008.124830,2007.214120,2006.303410,2005.392700,2006.186810,2006.980920,2007.775030,2008.569140,2009.363250,2010.157360,2010.951470,2011.745580,2012.539690,2013.333800,2015.809830,2018.285860,2020.761890,2023.237920,2025.713950,2028.189980,2030.666010,2033.142040,2035.618070,2038.094100,2041.669030,2045.243960,2048.818890,2052.393820,2055.968750,2059.543680,2063.118610,2066.693540,2070.268470,2073.843400,2078.502600,2083.161800,2087.821000,2092.480200,2097.139400,2101.798600,2106.457800,2111.117000,2115.776200,2120.435400,2127.831120,2135.226840,2142.622560,2150.018280,2157.414000,2164.809720,2172.205440,2179.601160,2186.996880,2194.392600,2202.829420,2211.266240,2219.703060,2228.139880,2236.576700,2245.013520,2253.450340,2261.887160,2270.323980,2278.760800,2288.771300,2298.781800,2308.792300,2318.802800,2328.813300,2338.823800,2348.834300,2358.844800,2368.855300,2378.865800,2388.876300,2402.083310,2415.290320,2428.497330,2441.704340,2454.911350,2468.118360,2481.325370,2494.532380,2507.739390,2520.946400,2534.23137272727,2547.51634545455,2560.80131818182,2574.08629090909,2587.37126363636,2600.65623636364,2613.94120909091,2627.22618181818,2640.51115454545,2653.79612727273,2667.081100,2684.935560,2702.790020,2720.644480,2738.498940,2756.353400,2774.207860,2792.062320,2809.916780,2827.771240,2845.625700,2864.05591818182,2882.48613636364,2900.91635454546,2919.34657272727,2937.77679090909,2956.20700909091,2974.63722727273,2993.06744545455,3011.49766363636,3029.92788181818,3048.358100,3068.08187272727,3087.80564545455,3107.52941818182,3127.25319090909,3146.97696363636,3166.70073636364,3186.42450909091,3206.14828181818,3225.87205454546,3245.59582727273,3265.319600,3297.02718181818,3328.73476363636,3360.44234545455,3392.14992727273,3423.85750909091,3455.56509090909,3487.27267272727,3518.98025454545,3550.68783636364,3582.39541818182,3614.103000,3630.01148181818,3645.91996363636,3661.82844545455,3677.73692727273,3693.64540909091,3709.55389090909,3725.46237272727,3741.37085454546,3757.27933636364,3773.18781818182,3789.096300,3815.94828181818,3842.80026363636,3869.65224545455,3896.50422727273,3923.35620909091,3950.20819090909,3977.06017272727,4003.91215454546,4030.76413636364,4057.61611818182,4084.468100,4111.340575,4138.213050,4165.085525,4191.958000,4218.830475,4245.702950,4272.575425,4299.447900,4326.320375,4353.192850,4380.065325,4406.937800,4438.52679090909,4470.11578181818,4501.70477272727,4533.29376363636,4564.88275454545,4596.47174545455,4628.06073636364,4659.64972727273,4691.23871818182,4722.82770909091,4754.416700,4789.99513636364,4825.57357272727,4861.15200909091,4896.73044545455,4932.30888181818,4967.88731818182,5003.46575454546,5039.04419090909,5074.62262727273,5110.20106363636,5145.779500,5180.94024166667,5216.10098333333,5251.261725,5286.42246666667,5321.58320833333,5356.743950,5391.90469166667,5427.06543333333,5462.226175,5497.38691666667,5532.54765833333,5567.708400,5608.562150,5649.415900,5690.269650,5731.123400,5771.977150,5812.830900,5853.684650,5894.538400,5935.392150,5976.245900,6017.099650,6057.953400,6098.40514166667,6138.85688333333,6179.308625,6219.76036666667,6260.21210833333,6300.663850,6341.11559166667,6381.56733333333,6422.019075,6462.47081666667,6502.92255833333,6543.374300,6596.24753333333,6649.12076666667,6701.994000,6754.86723333333,6807.74046666667,6860.613700,6913.48693333333,6966.36016666667,7019.233400,7072.10663333333,7124.97986666667,7177.853100,7242.966275,7308.079450,7373.192625,7438.305800,7503.418975,7568.532150,7633.645325,7698.758500,7763.871675,7828.984850,7894.098025,7959.211200,8019.461875,8079.712550,8139.963225,8200.213900,8260.464575,8320.715250,8380.965925,8441.216600,8501.467275,8561.717950,8621.968625,8682.219300,8746.099225,8809.979150,8873.859075,8937.739000,9001.618925,9065.498850,9129.378775,9193.258700,9257.138625,9321.018550,9384.898475,9448.778400,9495.26436923077,9541.75033846154,9588.23630769231,9634.72227692308,9681.20824615385,9727.69421538462,9774.18018461538,9820.66615384615,9867.15212307692,9913.63809230769,9960.12406153846,10006.6100307692,10053.096]
    psit = [0.000782,0.000846,0.00091,0.00095450,0.000999,0.0010195,0.00104,0.00103,0.00102,0.00098350,0.000947,0.00089750,0.000848,0.00080150,0.000755,0.000719,0.000683,0.00064950,0.000616,0.00057750,0.000539,0.000495,0.000451,0.000406,0.000361,0.000311,0.000261,0.000213,0.000165,0.00012815,9.13000000000000e-05,6.20000000000000e-05,3.27000000000000e-05,3.55000000000000e-06,-2.56000000000000e-05,-5.42500000000000e-05,-8.29000000000000e-05,-0.00011295,-0.000143,-0.00016950,-0.000196,-0.00021350,-0.000231,-0.00024,-0.000249,-0.00026750,-0.000286,-0.000309,-0.000332,-0.00033550,-0.000339,-0.00032850,-0.000318,-0.000293,-0.000268,-0.000218,-0.000168,-0.00010190,-3.58000000000000e-05,2.02500000000000e-05,7.63000000000000e-05,9.56500000000000e-05,0.000115,0.000112,0.000109,8.56500000000000e-05,6.23000000000000e-05,3.69500000000000e-05,1.16000000000000e-05,-1.26000000000000e-05,-3.68000000000000e-05,-6.25000000000000e-05,-8.82000000000000e-05,-0.00012860,-0.000169,-0.00021150,-0.000254,-0.00028850,-0.000323,-0.000342,-0.000361,-0.000372,-0.000383,-0.000396,-0.000409,-0.000426,-0.000443,-0.000447,-0.000451,-0.00044250,-0.000434,-0.000427,-0.00042,-0.000419,-0.000418,-0.000408,-0.000398,-0.00035250,-0.000307,-0.00023650,-0.000166,-0.00014150,-0.000117,-2.89500000000000e-05,5.91000000000000e-05,0.00017905,0.000299,0.000515,0.000731,0.00092050,0.00111,0.0013850,0.00166,0.0020250,0.00239,0.0027350,0.00308,0.0034750,0.00387,0.00425,0.00463,0.00499,0.00535,0.00576,0.00617,0.0065950,0.00702,0.00758,0.00814,0.0089850,0.00983,0.010865,0.0119,0.012950,0.014,0.014750,0.0155,0.015850,0.0162,0.016150,0.0161,0.0157,0.0153,0.0148,0.0143,0.0137,0.0131,0.012450,0.0118,0.011150,0.0105,0.00987,0.00924,0.0086150,0.00799,0.00744,0.00689,0.00624,0.00559,0.00503,0.00447,0.0039750,0.00348,0.0029350,0.00239,0.00187,0.00135,0.00089650,0.000443,2.99999999999999e-06,-0.000437,-0.00087850,-0.00132,-0.00168,-0.00204,-0.0024450,-0.00285,-0.00317,-0.00349,-0.0037650,-0.00404,-0.0042950,-0.00455,-0.0047950,-0.00504,-0.0052,-0.00536,-0.0055150,-0.00567,-0.0058050,-0.00594,-0.0060050,-0.00607,-0.0061150,-0.00616,-0.00616,-0.00616,-0.0061550,-0.00615,-0.0060850,-0.00602,-0.0058650,-0.00571,-0.00555,-0.00539,-0.0051750,-0.00496,-0.00469,-0.00442,-0.00413,-0.00384,-0.0035050,-0.00317,-0.00277,-0.00237,-0.0020050,-0.00164,-0.0012225,-0.000805,-0.0004049950000,-4.99000000000000e-06,0.0005375050000,0.00108,0.0017350,0.00239,0.00335,0.00431,0.00564,0.00697,0.0085350,0.0101,0.011550,0.013,0.0139,0.0148,0.015150,0.0155,0.015450,0.0154,0.014950,0.0145,0.0139,0.0133,0.0127,0.0121,0.0115,0.0109,0.010245,0.00959,0.0089450,0.0083,0.00752,0.00674,0.00616,0.00558,0.0048850,0.00419,0.00375,0.00331,0.0029950,0.00268,0.00212,0.00156,0.0010095,0.000459,-6.90000000000000e-05,-0.000597,-0.0010785,-0.00156,-0.00309,-0.00462,-0.00492,-0.00522,-0.0056,-0.00598,-0.00631,-0.00664,-0.00707,-0.0075,-0.0076550,-0.00781,-0.00798,-0.00815,-0.0083650,-0.00858,-0.0086750,-0.00877,-0.0088350,-0.0089,-0.00897,-0.00904,-0.00885,-0.00866,-0.0086650,-0.00867,-0.0084950,-0.00832,-0.00809,-0.00786,-0.0075,-0.00714,-0.00666,-0.00618,-0.0057950,-0.00541,-0.0045950,-0.00378,-0.0030450,-0.00231,-0.0014755,-0.000641,0.00053950,0.00172,0.0029950,0.00427,0.0057150,0.00716,0.00898,0.0108,0.012850,0.0149,0.017350,0.0198,0.0225,0.0252,0.0285,0.0318,0.0352,0.0386,0.042450,0.0463,0.050350,0.0544,0.058950,0.0635,0.067850,0.0722,0.0769,0.0816,0.086650,0.0917,0.096850,0.102,0.10750,0.113,0.11850,0.124,0.13050,0.137,0.14450,0.152,0.16,0.168,0.177,0.186,0.195,0.204,0.21250,0.221,0.228,0.235,0.24050,0.246,0.25,0.254,0.255,0.256,0.25450,0.253,0.24850,0.244,0.238,0.232,0.22350,0.215,0.206,0.197,0.18750,0.178,0.16750,0.157,0.14650,0.136,0.12550,0.115,0.10495,0.0949,0.084850,0.0748,0.065450,0.0561,0.046850,0.0376,0.029350,0.0211,0.013070,0.00504,-0.00205,-0.00914,-0.015370,-0.0216,-0.0276,-0.0336,-0.0387,-0.0438,-0.048350,-0.0529,-0.057050,-0.0612,-0.0644,-0.0676,-0.070550,-0.0735,-0.075850,-0.0782,-0.080150,-0.0821,-0.0836,-0.0851,-0.086150,-0.0872,-0.087950,-0.0887,-0.0889,-0.0891,-0.0892,-0.0893,-0.089250,-0.0892,-0.0887,-0.0882,-0.087650,-0.0871,-0.0862,-0.0853,-0.084350,-0.0834,-0.0823,-0.0812,-0.080050,-0.0789,-0.077750,-0.0766,-0.0752,-0.0738,-0.072550,-0.0713,-0.0699,-0.0685,-0.067250,-0.066,-0.064750,-0.0635,-0.062050,-0.0606,-0.059250,-0.0579,-0.056850,-0.0558,-0.0546,-0.0534,-0.0523,-0.0512,-0.050,-0.0488,-0.047850,-0.0469,-0.0459,-0.0449,-0.043850,-0.0428,-0.0419,-0.041,-0.040150,-0.0393,-0.0382,-0.0371,-0.036450,-0.0358,-0.034750,-0.0337,-0.0326,-0.0315,-0.030550,-0.0296,-0.028650,-0.0277,-0.026550,-0.0254,-0.024150,-0.0229,-0.021850,-0.0208,-0.019350,-0.0179,-0.016450,-0.015,-0.013550,-0.0121,-0.010165,-0.00823,-0.0066450,-0.00506,-0.0026950,-0.00033,0.00136,0.00305,0.0055550,0.00806,0.010880,0.0137,0.016450,0.0192,0.022450,0.0257,0.029350,0.033,0.036950,0.0409,0.045450,0.050,0.055450,0.0609,0.067150,0.0734,0.0813,0.0892,0.0986,0.108,0.11950,0.131,0.145,0.159,0.17550,0.192,0.212,0.232,0.254,0.276,0.302,0.328,0.35650,0.385,0.41450,0.444,0.47250,0.501,0.527,0.553,0.57350,0.594,0.60650,0.619,0.62450,0.63,0.626,0.622,0.61250,0.603,0.58750,0.572,0.55350,0.535,0.51350,0.492,0.469,0.446,0.424,0.402,0.37850,0.355,0.33350,0.312,0.29050,0.269,0.25,0.231,0.212,0.193,0.17550,0.158,0.142,0.126,0.11030,0.0946,0.080,0.0654,0.051550,0.0377,0.023070,0.00844,-0.00503,-0.0185,-0.031350,-0.0442,-0.056750,-0.0693,-0.081450,-0.0936,-0.10530,-0.117,-0.12850,-0.14,-0.15050,-0.161,-0.17150,-0.182,-0.191,-0.20,-0.20850,-0.217,-0.22550,-0.234,-0.241,-0.248,-0.25450,-0.261,-0.267,-0.273,-0.27850,-0.284,-0.28850,-0.293,-0.29750,-0.302,-0.30550,-0.309,-0.312,-0.315,-0.318,-0.321,-0.323,-0.325,-0.32650,-0.328,-0.329,-0.33,-0.33050,-0.331,-0.331,-0.331,-0.33050,-0.33,-0.329,-0.328,-0.327,-0.326,-0.32350,-0.321,-0.319,-0.317,-0.31450,-0.312,-0.30850,-0.305,-0.30150,-0.298,-0.294,-0.29,-0.28550,-0.281,-0.27650,-0.272,-0.267,-0.262,-0.25650,-0.251,-0.246,-0.241,-0.23550,-0.23,-0.22350,-0.217,-0.21150,-0.206,-0.20,-0.194,-0.18750,-0.181,-0.175,-0.169,-0.16250,-0.156,-0.15,-0.144,-0.138,-0.132,-0.126,-0.12,-0.114,-0.108,-0.10105,-0.0941,-0.0875,-0.0809,-0.074350,-0.0678,-0.0603,-0.0528,-0.055450,-0.0581,-0.0513,-0.0445,-0.035750,-0.027,-0.017735,-0.00847,0.0011150,0.0107,0.022950,0.0352,0.045250,0.0553,0.069350,0.0834,0.0967,0.11,0.125,0.14,0.157,0.174,0.19050,0.207,0.226,0.245,0.264,0.283,0.305,0.327,0.35050,0.374,0.39650,0.419,0.449,0.479,0.50850,0.538,0.564,0.59,0.621,0.652,0.685,0.718,0.752,0.786,0.82250,0.859,0.89650,0.934,0.972,1.01,1.0550,1.1,1.1450,1.19,1.24,1.29,1.34,1.39,1.44,1.49,1.5450,1.6,1.6650,1.73,1.81,1.89,1.97,2.05,2.1450,2.24,2.3450,2.45,2.5650,2.68,2.8150,2.95,3.1050,3.26,3.4550,3.65,3.8650,4.08,4.33,4.58,4.85,5.12,5.3850,5.65,5.9650,6.28,6.63,6.98,7.3150,7.65,8.0150,8.38,8.7650,9.15,9.8750,10.6,11.050,11.5,11.9,12.3,12.650,13,13.3,13.6,13.750,13.9,13.950,14,13.950,13.9,13.6,13.3,13.1,12.9,12.650,12.4,12,11.6,11.150,10.7,10.180,9.66,9.09,8.52,7.92,7.32,6.69,6.06,5.43,4.8,4.17,3.54,2.9350,2.33,1.7350,1.14,0.57441,0.00882,-0.52059,-1.05,-1.5450,-2.04,-2.5,-2.96,-3.3750,-3.79,-4.18,-4.57,-4.9250,-5.28,-5.6,-5.92,-6.22,-6.52,-6.7950,-7.07,-7.3350,-7.6,-7.8550,-8.11,-8.35,-8.59,-8.8050,-9.02,-9.22,-9.42,-9.6,-9.78,-9.94,-10.1,-10.250,-10.4,-10.5,-10.6,-10.650,-10.7,-10.8,-10.9,-10.950,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-10.950,-10.9,-10.850,-10.8,-10.750,-10.7,-10.650,-10.6,-10.550,-10.5,-10.450,-10.4,-10.3,-10.2,-10.1,-10,-9.93,-9.86,-9.78,-9.7,-9.61,-9.52,-9.4350,-9.35,-9.26,-9.17,-9.08,-8.99,-8.9,-8.81,-8.72,-8.63,-8.53,-8.43,-8.34,-8.25,-8.16,-8.07,-7.9850,-7.9,-7.8050,-7.71,-7.6250,-7.54,-7.4550,-7.37,-7.2850,-7.2,-7.1150,-7.03,-6.9550,-6.88,-6.7950,-6.71,-6.6350,-6.56,-6.4850,-6.41,-6.34,-6.27,-6.1950,-6.12,-6.0450,-5.97,-5.9150,-5.86,-5.79,-5.72,-5.6650,-5.61,-5.55,-5.49,-5.43,-5.37,-5.31,-5.25,-5.1950,-5.14,-5.0850,-5.03,-4.9850,-4.94,-4.8950,-4.85,-4.81,-4.77,-4.7150,-4.66,-4.62,-4.58,-4.5450,-4.51,-4.47,-4.43,-4.3950,-4.36,-4.3150,-4.27,-4.24,-4.21,-4.18,-4.15,-4.12,-4.09,-4.06,-4.03,-4.0050,-3.98,-3.9550,-3.93,-3.9050,-3.88,-3.85,-3.82,-3.8,-3.78,-3.7550,-3.73,-3.71,-3.69,-3.67,-3.65,-3.6250,-3.6,-3.5850,-3.57,-3.5450,-3.52,-3.5050,-3.49,-3.47,-3.45,-3.43,-3.41,-3.3950,-3.38,-3.36,-3.34,-3.3250,-3.31,-3.2950,-3.28,-3.26,-3.24,-3.22,-3.2,-3.19,-3.18,-3.16,-3.14,-3.1250,-3.11,-3.09,-3.07,-3.0550,-3.04,-3.0250,-3.01,-2.9950,-2.98,-2.96,-2.94,-2.9250,-2.91,-2.8950,-2.88,-2.86,-2.84,-2.8250,-2.81,-2.7950,-2.78,-2.76,-2.74,-2.73,-2.72,-2.6950,-2.67,-2.6550,-2.64,-2.6250,-2.61,-2.5950,-2.58,-2.5650,-2.55,-2.53,-2.51,-2.49,-2.47,-2.4550,-2.44,-2.4250,-2.41,-2.39,-2.37,-2.35,-2.33,-2.3150,-2.3,-2.28,-2.26,-2.24,-2.22,-2.2,-2.18,-2.1650,-2.15,-2.13,-2.11,-2.09,-2.07,-2.0550,-2.04,-2.0150,-1.99,-1.9750,-1.96,-1.94,-1.92,-1.9,-1.88,-1.86,-1.84,-1.8250,-1.81,-1.7850,-1.76,-1.75,-1.74,-1.7150,-1.69,-1.67,-1.65,-1.6350,-1.62,-1.6,-1.58,-1.5650,-1.55,-1.5350,-1.52,-1.4950,-1.47,-1.4550,-1.44,-1.42,-1.4,-1.3850,-1.37,-1.3550,-1.34,-1.3150,-1.29,-1.28,-1.27,-1.2450,-1.22,-1.2050,-1.19,-1.16,-1.13,-1.12,-1.11,-1.08,-1.05,-1.0230,-0.996,-0.97250,-0.949,-0.926,-0.903,-0.869,-0.835,-0.799,-0.763,-0.72250,-0.682,-0.64950,-0.617,-0.57450,-0.532,-0.48550,-0.439,-0.39550,-0.352,-0.307,-0.262,-0.197,-0.132,-0.0874,-0.0428,0.00835,0.0595,0.10775,0.156,0.20750,0.259,0.31550,0.372,0.41850,0.465,0.513,0.561,0.613,0.665,0.71050,0.756,0.79750,0.839,0.887,0.935,0.97750,1.02,1.06,1.1,1.1450,1.19,1.2250,1.26,1.3,1.34,1.3850,1.43,1.47,1.51,1.55,1.59,1.6350,1.68,1.72,1.76,1.8050,1.85,1.9,1.95,2.0050,2.06,2.11,2.16,2.2150,2.27,2.33,2.39,2.4550,2.52,2.6,2.68,2.7650,2.85,2.94,3.03,3.13,3.23,3.34,3.45,3.57,3.69,3.8150,3.94,4.1150,4.29,4.4950,4.7,4.9250,5.15,5.43,5.71,6.0150,6.32,6.66,7,7.4450,7.89,8.4150,8.94,9.62,10.3,11.1,11.9,12.850,13.8,14.850,15.9,17.050,18.2,19.5,20.8,22.3,23.8,25.4,27,28.7,30.4,32.1,33.8,36.050,38.3,40.3,42.3,43.9,45.5,46.850,48.2,49.2,50.2,51,51.8,52.3,52.8,52.850,52.9,52.550,52.2,51.7,51.2,50.2,49.2,47.7,46.2,44.4,42.6,40.550,38.5,36.150,33.8,31.3,28.8,26.3,23.8,21.150,18.5,16.2,13.9,11.485,9.07,6.85,4.63,2.5655,0.501,-1.3995,-3.3,-5.02,-6.74,-8.2950,-9.85,-11.225,-12.6,-13.8,-15,-16.1,-17.2,-18.150,-19.1,-19.950,-20.8,-21.6,-22.4,-23.050,-23.7,-24.350,-25,-25.5,-26,-26.450,-26.9,-27.3,-27.7,-28.050,-28.4,-28.7,-29,-29.250,-29.5,-29.750,-30,-30.150,-30.3,-30.450,-30.6,-30.750,-30.9,-31,-31.1,-31.150,-31.2,-31.250,-31.3,-31.350,-31.4,-31.4,-31.4,-31.4,-31.4,-31.4,-31.4,-31.350,-31.3,-31.250,-31.2,-31.1,-31,-30.9,-30.8,-30.7,-30.6,-30.5,-30.4,-30.3,-30.2,-30.050,-29.9,-29.750,-29.6,-29.450,-29.3,-29.150,-29,-28.8,-28.6,-28.4,-28.2,-28,-27.8,-27.6,-27.4,-27.2,-27,-26.8,-26.6,-26.350,-26.1,-25.6,-25.1,-24.9,-24.7,-24.5,-24.3,-24.050,-23.8,-23.550,-23.3,-23.1,-22.9,-22.650,-22.4,-22.150,-21.9,-21.7,-21.5,-21.250,-21,-20.750,-20.5,-20.250,-20,-19.750,-19.5,-19.3,-19.1,-18.850,-18.6,-18.350,-18.1,-17.9,-17.7,-17.450,-17.2,-16.950,-16.7,-16.5,-16.3,-16.050,-15.8,-15.6,-15.4,-15.2,-15,-14.750,-14.5,-14.3,-14.1,-13.9,-13.7,-13.5,-13.3,-13.1,-12.9,-12.7,-12.5,-12.3,-12.1,-11.9,-11.7,-11.550,-11.4,-11.2,-11,-10.8,-10.6,-10.450,-10.3,-10.115,-9.93,-9.7650,-9.6,-9.4350,-9.27,-9.11,-8.95,-8.7950,-8.64,-8.4950,-8.35,-8.2,-8.05,-7.9,-7.75,-7.61,-7.47,-7.3350,-7.2,-7.0550,-6.91,-6.78,-6.65,-6.53,-6.41,-6.2850,-6.16,-6.0250,-5.89,-5.7650,-5.64,-5.5250,-5.41,-5.29,-5.17,-5.0450,-4.92,-4.81,-4.7,-4.6050,-4.51,-4.3950,-4.28,-4.1650,-4.05,-3.96,-3.87,-3.78,-3.69,-3.5850,-3.48,-3.38,-3.28,-3.19,-3.1,-3.0150,-2.93,-2.85,-2.77,-2.6850,-2.6,-2.51,-2.42,-2.3450,-2.27,-2.1950,-2.12,-2.0450,-1.97,-1.8950,-1.82,-1.7550,-1.69,-1.62,-1.55,-1.4750,-1.4,-1.33,-1.26,-1.2050,-1.15,-1.1,-1.05,-0.98,-0.91,-0.81650,-0.723,-0.66650,-0.61,-0.56350,-0.517,-0.473,-0.429,-0.37950,-0.33,-0.298,-0.266,-0.23550,-0.205,-0.176,-0.147,-0.11520,-0.0834,-0.058550,-0.0337,-0.00355,0.0266,0.0658,0.105,0.149,0.193,0.24450,0.296,0.34550,0.395,0.44650,0.498,0.553,0.608,0.661,0.714,0.76450,0.815,0.866,0.917,0.96350,1.01,1.0650,1.12,1.16,1.2,1.24,1.28,1.32,1.36,1.39,1.42,1.4550,1.49,1.52,1.55,1.58,1.61,1.6350,1.66,1.69,1.72,1.74,1.76,1.7850,1.81,1.8350,1.86,1.89,1.92,1.9450,1.97,1.9950,2.02,2.0450,2.07,2.09,2.11,2.14,2.17,2.2,2.23,2.2550,2.28,2.3,2.32,2.3450,2.37,2.3950,2.42,2.44,2.46,2.48,2.5,2.52,2.54,2.5550,2.57,2.5850,2.6,2.6150,2.63,2.65,2.67,2.6850,2.7,2.7050,2.71,2.73,2.75,2.76,2.77,2.78,2.79,2.7950,2.8,2.81,2.82,2.8250,2.83,2.8350,2.84,2.8450,2.85,2.8550,2.86,2.86,2.86,2.86,2.86,2.86,2.86,2.8650,2.87,2.86,2.85,2.8550,2.86,2.86,2.86,2.8550,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.85,2.8550,2.86,2.86,2.86,2.86,2.86,2.8650,2.87,2.8750,2.88,2.89,2.9,2.91,2.92,2.93,2.94,2.95,2.96,2.9750,2.99,3.0050,3.02,3.04,3.06,3.08,3.1,3.12,3.14,3.17,3.2,3.2250,3.25,3.2850,3.32,3.35,3.38,3.4150,3.45,3.5,3.55,3.59,3.63,3.68,3.73,3.7650,3.8,3.86,3.92,3.9750,4.03,4.09,4.15,4.22,4.29,4.35,4.41,4.4850,4.56,4.6150,4.67,4.74,4.81,4.9550,5.1,5.2850,5.47,5.43,5.39,5.4550,5.52,5.5950,5.67,5.79,5.91,6.0050,6.1,6.1850,6.27,6.4,6.53,6.62,6.71,6.8350,6.96,7.09,7.22,7.3050,7.39,7.51,7.63,7.8150,8,8.0850,8.17,8.29,8.41,8.5450,8.68,8.83,8.98,9.1250,9.27,9.4450,9.62,9.76,9.9,9.95,10,10.3,10.6,10.650,10.7,10.9,11.1,11.3,11.5,11.650,11.8,12,12.2,12.4,12.6,12.750,12.9,13.150,13.4,13.6,13.8,14,14.2,14.450,14.7,14.950,15.2,15.450,15.7,16,16.3,16.550,16.8,17.1,17.4,17.7,18,18.3,18.6,18.950,19.3,19.650,20,20.350,20.7,21.1,21.5,21.850,22.2,22.6,23,23.450,23.9,24.350,24.8000000000000]

    count = 0
    for wavelen in nm_sample:
        absorp[str(wavelen)] = [asw[count], psit[count]]
        count += 1

    #Select the water absorption constants in function of the input wavelength
    aswlist = []
    for nm in nm_touse:
        constants = absorp[str(nm)]

        psitvalue = constants[1]

        aswvalue = constants[0]

        PSIT = np.full(temp.shape, psitvalue).astype(np.float32)
        ASW = np.full(temp.shape, aswvalue).astype(np.float32)
        #Calculate asw corrected values for each band
        asw_corr = ASW + PSIT * ((temp + 273.15) - (22 + 273.15))
        aswlist.append(asw_corr)
        psiT = None  # Clear psiT
        ASW  = None  # Clear psiT
    asw_Tcorr = np.stack(aswlist, axis=2)
    return asw_Tcorr

In [9]:
def Ucal(rrs, Lconstants):
#-------------------------------------------------------------------------
#   Calculates the dimensionless value u(λ) derived from the water's inherent optical properties.
#
# INPUTS:
#
#   rrs      - Remote Sensing Reflectance measured for a given band
#   L constants - defined by Gordon et al (1988)
#
# OUTPUTS:
#
#  Ues - Estimated u(λ) values
#--------------------------------------------------------------------------
    delta_root = np.sqrt((Lconstants[0])**2 - (4 * Lconstants[1] * (-rrs)))

    delta_root[np.imag(delta_root) != 0] = np.nan


    Ues = (-Lconstants[0] + delta_root)/(2*Lconstants[1])
    return Ues

In [10]:
def Weight(rrs, SPM_model, Lconstant, Uestimate, wavelengths, iop_median):
# Weight assessment function for SPM and associated uncertainty estimates
#
# INPUTS:
#
# rrs        -  measured below water rrs(nm)
# Uestimate  -  function of IOPs by Gordon et al (1988)
# Lconstant  -  constants to calculate U by Gordon et al (1988)
# SPM_model  -  all SPM estimates
# wavelengths-  wavelengths associated with measured Remote sensing reflectance
#
# OUTPUTS:
#
# Weight     - Maximum uncertainty taking into account absolute and relative uncertainties propagated to SPM
#
#---------------------------------------------------------------------------------------------------------------------------------------------------
    SPM_median = np.nanmedian(SPM_model, axis=2)

    def calculate_std(arr):
        return np.nanstd(arr)

    max_noise = np.zeros((rrs.shape[0], rrs.shape[1], len(wavelengths)))
    deltaU = np.zeros((rrs.shape[0], rrs.shape[1], len(wavelengths)))

    for i in range(len(wavelengths)):
        relative = np.sqrt(2)*0.05*rrs[:,:,i]

        absolute = generic_filter(rrs[:,:,i], calculate_std, size=(3, 3))

        max_noise[:,:,i] = np.maximum(relative, absolute)

        deltaU[:,:,i] = max_noise[:,:,i]/ (Lconstant[0]+(2*Lconstant[1]*Uestimate[:,:,i]))

    #Final weight

    weight = 1 / ((deltaU * SPM_median) / ((Uestimate - (np.square(Uestimate)) * iop_median)))

    weight = weight.astype(np.float32)

    return weight

In [11]:
def delete_files(file_list):
#    Deletes all the files from the given list of file paths.
#
#    INPUTS:
#
#    file_list (list): A list of file paths to be deleted.
#    OUTPUTS:
#
#    None
#---------------------------------------------------------------------------------------------------------------------------------------------------
    for file_path in file_list:
        try:
            os.remove(file_path)
            print(f"Temporal file {file_path} has been deleted successfully")
        except FileNotFoundError:
            print(f"Temporal file {file_path} does not exist")
        except Exception as e:
            print(f"An error occurred while deleting {file_path}: {e}")

In [12]:
def get_rrs_and_temperature_gee_data(geo_file, bandlist):
#    Reads the .nc file containing the rrs bands of the satellite image and use
#    the lat/lon info to get the temperature from the Google Earth Engine collection
#    GCOM-C/SGLI L3 Sea Surface Temperature (V3) https://developers.google.com/earth-engine/datasets/catalog/JAXA_GCOM-C_L3_OCEAN_SST_V3
#
#    INPUTS:
#
#    geo_file (file): Address to the .nc satellite image.
#    bandlist (list): A list with the bandnames to be read from your .nc file
#
#    OUTPUTS:
#    rrs(np matrix): bands to be processed by the SPM MW algorithm
#    SST: Sea Surface Temperature
#---------------------------------------------------------------------------------------------------------------------------------------------------

    geo_fid = xr.open_dataset(geo_file)

    lon = geo_fid.lon.data
    lat = geo_fid.lat.data

    #Get AOI bounding box
    min_lat = np.min(lat)
    max_lat = np.max(lat)

    min_lon = np.min(lon)
    max_lon = np.max(lon)


    #Read rrs bands
    rrs_to_stack = []

    for band in bandlist:
        rrs_to_stack.append(geo_fid[band])

    #Stack bands in a single rrs file
    rrs = np.dstack(rrs_to_stack)


    # Extract the date from the geo_file's global attribute 'isodate'.
    isodate = geo_fid.attrs['isodate']
    date = datetime.fromisoformat(isodate)

    # Define the area of interest.
    aoi = ee.Geometry.Rectangle([float(min_lon), float(min_lat), float(max_lon), float(max_lat)])

    # Calculate the sea surface temperature in kelvin degrees.
    SSTgee = (ee.ImageCollection('JAXA/GCOM-C/L3/OCEAN/SST/V3')
              .filterDate((date + timedelta(days=-2)).strftime('%Y-%m-%d'),(date + timedelta(days=3)).strftime('%Y-%m-%d'))
              .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D'))
              .select(['SST_AVE'])
              .mean()
              .clip(aoi)
              .multiply(0.0012)
              .add(-10))

    # Calculate the median temperature of the water body to fulfill the lacking data.
    median_temperature_rq = SSTgee.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=aoi,
        scale=300  # Set the scale according to your needs.
    )

    median_temperature = float(median_temperature_rq.getInfo()['SST_AVE'])

    # Apply to the nodata mask.
    nodatamask = SSTgee.select(['SST_AVE']).mask().clip(aoi)

    temperature = (SSTgee.reproject(crs='EPSG:4326', scale=300)
                    .updateMask(nodatamask)
                    .unmask(median_temperature))

    # Get the download URL for the temperature data.
    tempurl = temperature.getDownloadUrl({
        'region': aoi,
        'scale': 300,
        'format': 'NPY'
    })
    responsetemp = requests.get(tempurl)

    # Load the temperature data.
    SST = np.load(io.BytesIO(responsetemp.content))['SST_AVE']

    geo_fid.close()

    return rrs, SST

In [13]:
def get_rrs_and_temperature_s3_data(geo_file, bandlist):
#    Reads the .nc file containing the rrs bands of the satellite image and use
#    the lat/lon info to get the temperature from the Copernicus DataSpace Ecosystem collection
#    Sentinel-3 SLSTR L1B collection https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S3SLSTR.html
#
#    INPUTS:
#
#    geo_file (file): Address to the .nc satellite image.
#    bandlist (list): A list with the bandnames to be read from your .nc file
#
#    OUTPUTS:
#    rrs(np matrix): bands to be processed by the SPM MW algorithm
#    SST: Sea Surface Temperature
#---------------------------------------------------------------------------------------------------------------------------------------------------

    geo_fid = xr.open_dataset(geo_file)

    lon = geo_fid.lon.data
    lat = geo_fid.lat.data

    #Get AOI bounding box
    min_lat = np.min(lat)
    max_lat = np.max(lat)

    min_lon = np.min(lon)
    max_lon = np.max(lon)


    #Read rrs bands
    rrs_to_stack = []

    for band in bandlist:
        rrs_to_stack.append(geo_fid[band])

    #Stack bands in a single rrs file
    rrs = np.dstack(rrs_to_stack)

    # Extract the date from the geo_file's global attribute 'isodate'.
    isodate = geo_fid.attrs['isodate']
    date = datetime.fromisoformat(isodate)

    # Define the area of interest.
    spatial_extent = {"west": float(min_lon), "east": float(max_lon), "south": float(min_lat), "north": float(max_lat)}
    temporal_extent = [(date + timedelta(days=-1)).strftime('%Y-%m-%d'), (date + timedelta(days=1)).strftime('%Y-%m-%d')]

    datacube = connection.load_collection(
        "SENTINEL3_SLSTR",   # Sentinel-3 SLSTR L1B collection
        spatial_extent=spatial_extent,
        temporal_extent=temporal_extent,
        bands=["S8"]              # Use the S8 band for water temperature
    )

    # Calculate the temporal average (reduce across the time dimension)
    datacube_avg = datacube.reduce_dimension(dimension="t", reducer="mean")

    output_file = "S3A_OLCI_water_temperature.tif"

    datacube_avg.download(output_file)

    with rasterio.open(output_file) as src:
        data_array = src.read(1)  # Read the first band (temperature)

    SST = data_array - 273.15 # Convert temperature to Celcius degrees

    delete_files([output_file])

    geo_fid.close()

    return rrs, SST

In [14]:
def MW_ALGORITHM(nm, rrs, L, S, Y, a_nap443, a_nap750, bbp700, SST, Q_threshold, imname):
# MW algorithm and for SPM and associated uncertainty estimates
#
# Original version by Juliana Tavora, University of Maine, 2020
# Python version by Sergio Molano Cardenas, Universidade Federal do Rio Grande do Sul, 2024
#
#
# INPUTS:
#
# nm          -  wavelengths associated with measured Remote sensing reflectance
# rrs         -  numpy matrix containing measured below water rrs for a satellite sensor(nm)
# U           -  function of IOPs by Gordon et al (1988)
# L           -  constants to calculate U by Gordon et al (1988)
# S           -  exponent of exponential spectral shape for a_NAP
# Y           -  exponent of power-law spectral shape for bbp
# a_nap443    -  mass specific absorption at the reference at 443 nm
# a_nap750    -  mass specific absorption at the reference at 750 nm
# bbp700      -  mass specific backscattering at 700 nm
# SST         -  temperature of the water in Celsius which remote sensing
#                reflectance was measured
# Q_threshold -  saturation threshold
# imname      -  name of the image being processed
#
# This function calls other 3 functions:
#    1) 'asw_correction' Water absorption for temperature correction
#    2) 'Weight' for estimates of maximum uncertainties and
#       establishing the weight used for final SPM and uncertainty estimates
#
# OUTPUTS:
#
# SPM_mw    -  SPM estimates
# err_mw    -  uncertainties associated with the method in percentage
#
# The SPM retrivals and uncertainties of this version of the algorithm
# are mainly oriented remote sensing data such as Sentinel3/OLCI,
# Landsat8/OLI, MODIS/aqua and others.
# Adjustments must be done however to the 'weight_assess' function

#-------------------------------------------------------------------------#
    dim = (len(S)*len(Y)*len(a_nap443)*len(a_nap750)*len(bbp700))

    #Resample the temperature to match the rrs data
    targ_dim0 = rrs.shape[0]

    targ_dim1 = rrs.shape[1]

    sst_dim0 = SST.shape[0]

    sst_dim1 = SST.shape[1]

    R_SST = ndimage.zoom(SST, (targ_dim0/sst_dim0, targ_dim1/sst_dim1), order=1)

    R_SST = R_SST.astype(np.float32)

    #Calculate U values for each rrs
    U = Ucal(rrs, L)

    #Generate water absorption IOP vectors
    dtype = np.float32
    shape_a_sw = (R_SST.shape[0], R_SST.shape[1], dim, len(nm))
    a_sw = np.memmap('a_sw.dat', dtype=dtype, mode='w+', shape=shape_a_sw)

    absorpw = aswcorrection(R_SST, nm).astype(np.float32)

    np.copyto(a_sw, np.transpose(np.repeat(absorpw[:, np.newaxis, :], dim, axis=1), (0, 2, 1, 3)))

    a_sw.flush()

    #Create Zeros arrays to store the processing data
    num_S = len(S)
    num_nm = len(nm)
    num_a_nap443 = len(a_nap443)
    num_a_nap750 = len(a_nap750)
    a_p = np.zeros((num_nm, num_a_nap443, num_a_nap750, num_S), dtype=np.float32)
    S_slope = np.zeros((num_nm, num_a_nap443, num_a_nap750, num_S), dtype=np.float32)
    a_nap443_matrix = np.zeros((num_nm, num_a_nap443, num_a_nap750, num_S), dtype=np.float32)
    a_nap750_matrix = np.zeros((num_nm, num_a_nap443, num_a_nap750, num_S), dtype=np.float32)


    for i in range(num_S):
        for ii in range(num_nm):
            for iii in range(num_a_nap443):
                for iiii in range(num_a_nap750):
                    a_p[ii, iii, iiii, i] = a_nap443[iii] * (np.exp(-S[i] * (nm[ii] - 443)) - np.exp(-S[i] * (750 - 443))) + a_nap750[iiii]
                    S_slope[ii, iii, iiii, i] = S[i]
                    a_nap443_matrix[ii, iii, iiii, i] = a_nap443[iii]
                    a_nap750_matrix[ii, iii, iiii, i] = a_nap750[iiii]
    # Reshape arrays
    ap = a_p.reshape((len(nm), -1)).T
    S_res = S_slope.reshape((len(nm), -1)).T
    a_nap443_res = a_nap443_matrix.reshape((len(nm), -1)).T
    a_nap750_res = a_nap750_matrix.reshape((len(nm), -1)).T

    # Clear variable a_p
    #a_p = None
    i = None
    ii = None
    iii = None
    iiii= None

    #Generate water backscattering IOP vectors
    num_Y = len(Y)
    num_nm = len(nm)
    num_bbp700 = len(bbp700)

    bb_p = np.zeros((num_nm, num_bbp700, num_Y))
    Y_slope = np.zeros((num_nm, num_bbp700, num_Y))
    bbp_matrix = np.zeros((num_nm, num_bbp700, num_Y))

    for n in range(num_Y):
        for nn in range(num_nm):
            for nnn in range(num_bbp700):
                bb_p[nn, nnn, n] = bbp700[nnn] * ((700 / nm[nn]) ** Y[n])
                Y_slope[nn, nnn, n] = Y[n]
                bbp_matrix[nn, nnn, n] = bbp700[nnn]

    # Reshape arrays
    bbp = bb_p.reshape((num_nm, -1)).T
    Y_res = Y_slope.reshape((num_nm, -1)).T
    bbp_res = bbp_matrix.reshape((num_nm, -1)).T

    # Clear variables
    #bb_p = None
    n = None
    nn = None
    nnn = None

    #Create IOP MATRIXES

    a_combinations_iop = len(S) * len(a_nap443) * len(a_nap750)
    b_combinations_iop = len(Y) * len(bbp700)

    # Initialize matrices
    a_combinations_iop = len(S) * len(a_nap443) * len(a_nap750)
    b_combinations_iop = len(Y) * len(bbp700)

    # Initialize matrices
    IOP_matrix = np.zeros((2, num_nm, dim), dtype=np.float32)
    shape_matrix = np.zeros((2, num_nm, dim), dtype=np.float32)
    coefs_matrix = np.zeros((3, num_nm, dim), dtype=np.float32)

    print(f"Creating IOP matrix")
    k = 0
    for i in range(a_combinations_iop):
        for n in range(b_combinations_iop):
            IOP_matrix[:, :, k] = np.vstack((ap[i, :], bbp[n, :]))
            shape_matrix[:, :, k] = np.vstack((S_res[i, :], Y_res[n, :]))
            coefs_matrix[:, :, k] = np.vstack((a_nap443_res[i, :], a_nap750_res[i, :], bbp_res[n, :]))
            k += 1

    shape_U_model = (rrs.shape[2], dim)
    shape_SPM_model = (rrs.shape[0],rrs.shape[1], dim,rrs.shape[2])
    shape_Q = (rrs.shape[0],rrs.shape[1], dim,rrs.shape[2])


    ## SPM calculation
    # Create memory-mapped arrays with a temporary file
    SPM_model = np.memmap('SPM_model.dat', dtype=dtype, mode='w+', shape=shape_SPM_model)
    Q = np.memmap('Q.dat', dtype=dtype, mode='w+', shape=shape_Q)

    # Calculate U_model directly on the memmap
    U_model = np.squeeze(IOP_matrix[1, :, :]) / (np.squeeze(IOP_matrix[0, :, :]) + np.squeeze(IOP_matrix[1, :, :]))

    print(f"Modeling SPM concentrations for image {imname}")
    for ii in range(rrs.shape[2]):
        for ix in range(rrs.shape[0]):
            for iy in range(rrs.shape[1]):
                SPM_model[ix, iy, :, ii] = (1.0 / np.squeeze(IOP_matrix[1, ii, :])) * ((np.squeeze(a_sw[ix,iy,:, ii])) / (((1 - U[ix,iy, ii]) / U[ix, iy, ii]) - (np.squeeze(IOP_matrix[0, ii, :]) / np.squeeze(IOP_matrix[1, ii, :]))))
                Q[ix, iy,:, ii] = U[ix,iy, ii] / U_model[ii, :]

    #Apply the Q saturation filter
    SPM_model[(SPM_model < 0) | (Q < 0) | (Q > Q_threshold)] = np.nan

    SPM_model.flush()
    Q.flush()

    ##CALCULATE IOP MEDIAN

    iop_simple = ((np.squeeze(IOP_matrix[0, :, :]) + np.squeeze(IOP_matrix[1, :, :])) / np.squeeze(IOP_matrix[1, :, :])).T

    iop_rep = np.tile(iop_simple[:,:, np.newaxis, np.newaxis], (1, 1,rrs.shape[0],rrs.shape[1] ))

    iop = np.transpose(iop_rep, (2, 3, 0, 1))

    iop[(SPM_model < 0) | (Q < 0) | (Q > Q_threshold)] = np.nan

    iop_median = np.nanmedian(iop, axis=2)

    #Weight estimates
    print(f"Calculating weights...")
    weight = Weight(rrs, SPM_model, L, U, nm, iop_median)

    shape_Wtemp = weight.shape + (dim,)  # This adds the 'dim' as a new axis
    shape_W = (weight.shape[0], weight.shape[1], dim, weight.shape[2])

    # Create memory-mapped arrays with a temporary file
    Wtemp = np.memmap('Wtemp.dat', dtype=dtype, mode='w+', shape=shape_Wtemp)
    W = np.memmap('W.dat', dtype=dtype, mode='w+', shape=shape_W)

    np.copyto(Wtemp, np.tile(weight[:, :, :, np.newaxis], (1, 1, 1, dim)))

    # Transpose Wtemp to get W
    np.copyto(W, np.transpose(Wtemp, (0, 1, 3, 2)))

    Wtemp.flush()
    W.flush()

    #Filtering the results for saturation

    W[(SPM_model < 0) | (Q < 0) | (Q > Q_threshold)] = np.nan
    Q[(SPM_model < 0) | (Q < 0) | (Q > Q_threshold)] = np.nan

    #Determing Degrees of Freedom of rrs spectra
    rrs_reshaped = rrs.reshape(-1, rrs.shape[-1])
    area_under_curve = np.trapz(rrs_reshaped, nm, axis=1) # Integrating along the spectra (2nd dimension)
    X = rrs_reshaped / area_under_curve[:, np.newaxis]  # Normalizing rrs by the 'area under the curve'

    X_no_nan = X[~np.isnan(X).any(axis=1)]

    X = X[~np.isnan(X).any(axis=1)]

    # Applying PCA
    pca = PCA()
    pca.fit(X_no_nan)

    # Getting the explained variance
    explained = pca.explained_variance_ratio_ * 100

    DoF = np.sum(explained > 1)  # Adjust this threshold based on the percentage of variance needed

    M = DoF


    a_sw.flush()
    Q.flush()

    #Delete temporal files
    del a_sw, IOP_matrix, shape_matrix, coefs_matrix, U_model, Q, Wtemp,
    files_to_delete1 = ['a_sw.dat', 'Q.dat', 'Wtemp.dat']
    delete_files(files_to_delete1)

    #SPM retrieval
    print(f"Multiplicative weigthed SPM concentration for {imname}")
    SPM_mw = np.nansum(np.nansum((SPM_model*W), axis=2), axis = 2) / np.nansum(np.nansum(W, axis=2), axis=2)

    SPM_84 = np.percentile(SPM_model, 84, axis=2)
    SPM_16 = np.percentile(SPM_model, 16, axis=2)

    SPM_max = (np.nansum((SPM_84*weight), axis=2) / (np.nansum(weight, axis=2)))*(1/np.sqrt(M))
    SPM_min = (np.nansum((SPM_16*weight), axis=2) / (np.nansum(weight, axis=2)))*(1/np.sqrt(M))

    err_mw = (((SPM_max - SPM_min) / 2) * 100) / SPM_mw

    ## Delete all temporary files
    SPM_model.flush()
    W.flush()

    del SPM_model, W

    files_to_delete2 = ['SPM_model.dat', 'W.dat']
    delete_files(files_to_delete2)

    return SPM_mw, err_mw

In [15]:
def write_results_netcdf(geo_file, SPM_mw, err_mw):
#    Write the SPM_mw estimates and its associated errors to the original .nc
#    satellite file
#
#    INPUTS:
#
#    geo_file (file): Address to the .nc satellite image.
#    SPM_mw (np matrix): numpy matrix containing the SPM MW estiamtes
#    err_mw (np matrix): numpy matrix containing the uncertainties associated to
#    the SPM estimates
#
#    OUTPUTS:
#    None
#---------------------------------------------------------------------------------------------------------------------------------------------------
     with Dataset(geo_file, mode='a', format='NETCDF4_CLASSIC') as ncfile:
        # Define dimensions
        ncfile.createDimension('dim1', SPM_mw.shape[0])
        ncfile.createDimension('dim2', SPM_mw.shape[1])

        # Create variables
        SPM_mw_var = ncfile.createVariable('SPM_mw', np.float32, ('dim1', 'dim2'))
        err_mw_var = ncfile.createVariable('err_mw', np.float32, ('dim1', 'dim2'))

        # Assign data to variables
        SPM_mw_var[:, :] = SPM_mw
        err_mw_var[:, :] = err_mw

        # Add units
        SPM_mw_var.units = 'mg/l'
        err_mw_var.units = '%'

## Set IOP coefficients

Set the variable for IOPs that you want to use for the SPM estimations.

In [16]:
#Set constants and variables, default bands adjusted for Sentinel-3 OLCI
# S           -  exponent of exponential spectral shape for non-algal particles absorption
# Y           -  exponent of power-law spectral shape for particulate matter backscattering
# a_nap443    -  mass specific absorption for non algal particles at the reference at 443 nm
# a_nap750    -  mass specific absorption for non algal particles at the reference at 750 nm
# bbp700      -  mass specific backscattering for particulate matter at 700 nm
#------------------------------------------------------------------------------------------------------------------------------------

#Shape variables
S = [0.003,0.004,0.005]
Y = np.arange(0, 1.9, 0.1).round(3).tolist()

#Mass-specific coefficients
a_nap443 = [0.05,0.06]
a_nap750 = [0.013,0.014]
bbp700 = [0.006,0.007,0.008,0.009,0.01,0.011,0.012,0.013,0.016,0.019,0.021,0.022]

L = [0.0949,0.0794] #Gordon et al., 1988 constants

#Insert the band wavelengths to work in nanometers (do it in ascending order)
#It is recommended to use bands between 630 to 670 nanometers and 700 to 2500 nanometers
nm = [665, 865]

#Insert the rrs band names matching the previously defined wavelengths and the
#exact name they have in your .nc satellite file
bandlist = ["Rrs_665", "Rrs_865"]

#Saturation threshold
Q_threshold  = 0.5

## Run MW algorithm

### Google drive connection (only if using Google Colab)

In [17]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Executing the MW algorithm for several files stored in a folder

In [ ]:
#Script example for executing the MW algorithm for several files
#
##The input files must be in format NedCDF .nc format and must containg valid
##rrs bands and stored in the folder path you will provide
#
##The resulting SPM will be written to the original .nc files
#-------------------------------------------------------------------------------------------------------------------------------

#Your Google drive folder containing the .nc satellite images with the rrs bands
#Path to your local folder if executing using Jupyter Notebook
folder_path = '/your_folder_path'

start_time = time.time()

for file in os.listdir(folder_path):
    print(f"Starting processing of file {file}")

    geo_file = os.path.join(folder_path, file)
    #select temperature data origin
    if DATA_SOURCE == 'OPENEO':
        rrs,SST = get_rrs_and_temperature_s3_data(geo_file, bandlist)

    elif DATA_SOURCE == 'GEE':
        rrs,SST = get_rrs_and_temperature_gee_data(geo_file, bandlist)
    else:
        print("No valid temperature data source")

    SPM_mw,err_mw = MW_ALGORITHM(nm, rrs, L, S, Y, a_nap443, a_nap750, bbp700, SST, Q_threshold, file)

    write_results_netcdf(geo_file, SPM_mw, err_mw)


end_time = time.time()  # End the timer
duration = (end_time - start_time) / 60

print(f"All files processed in {duration:.2f} minutes")